## This notebook will help you train a vanilla Point-Cloud AE with the basic architecture we used in our paper.
    (it assumes latent_3d_points is in the PYTHONPATH and the structural losses have been compiled)

In [7]:
#Setup environment to be compatible with colab
!git clone https://github.com/T60D/latent_3d_points.git
%tensorflow_version 1.x
import sys
import tensorflow
print(tensorflow.__version__)

!apt-get install -qq gcc-5 g++-5 -y &> /dev/null
!ln -s /usr/bin/gcc-5 &> /dev/null
!ln -s /usr/bin/g++-5 &> /dev/null

!sudo apt-get update &> /dev/null
!sudo apt-get upgrade &> /dev/null

%cd /content/latent_3d_points/external/structural_losses
!make

fatal: destination path 'latent_3d_points' already exists and is not an empty directory.
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.3`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.2
/content/latent_3d_points/external/structural_losses
make: Circular tf_approxmatch_g.cu <- tf_approxmatch_g.cu.o dependency dropped.
make: Circular tf_nndistance_g.cu <- tf_nndistance_g.cu.o dependency dropped.
make: Nothing to be done for 'all'.


In [6]:
#Import drive where the files are stored
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/

import os.path as osp

from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from latent_3d_points.src.tf_utils import reset_tf_graph
from latent_3d_points.src.general_utils import plot_3d_point_cloud

/content






External Losses (Chamfer-EMD) were not loaded.
External Losses (Chamfer-EMD) cannot be loaded. Please install them first.


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Define Basic Parameters

In [5]:
top_out_dir = '/content/drive/Shareddrives/CS230/'          # Use to save Neural-Net check-points etc.
top_in_dir = '/content/drive/Shareddrives/CS230/dev_data' # Top-dir of where point-clouds are stored.

experiment_name = 'single_class_ae'
n_pc_points = 2048                # Number of points per model.
bneck_size = 128                  # Bottleneck-AE size
ae_loss = 'chamfer'                   # Loss to optimize: 'emd' or 'chamfer'
class_name = "plane"

Load Point-Clouds

In [14]:
#syn_id = snc_category_to_synth_id()[class_name]
#class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder("/content/drive/Shareddrives/CS230/dev_data", n_threads=8, file_ending='.ply', verbose=True)

KeyError: ignored

Load default training parameters (some of which are listed beloq). For more details please print the configuration object.

    'batch_size': 50   
    
    'denoising': False     (# by default AE is not denoising)

    'learning_rate': 0.0005

    'z_rotate': False      (# randomly rotate models of each batch)
    
    'loss_display_step': 1 (# display loss at end of these many epochs)
    'saver_step': 10       (# over how many epochs to save neural-network)

In [10]:
train_params = default_train_params()

In [12]:
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)
train_dir = create_dir(osp.join(top_out_dir, experiment_name))

In [17]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5   # How often to evaluate/print out loss on 
                         # held_out data (if they are provided in ae.train() ).
conf.save(osp.join(train_dir, 'configuration'))

TypeError: ignored

If you ran the above lines, you can reload a saved model like this:

In [15]:
load_pre_trained_ae = False
restore_epoch = 500
if load_pre_trained_ae:
    conf = Conf.load(train_dir + '/configuration')
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)
    ae.restore_model(conf.train_dir, epoch=restore_epoch)

Build AE Model.

In [16]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)






Building Encoder


NameError: ignored

Train the AE (save output to train_stats.txt) 

In [ ]:
buf_size = 1 # Make 'training_stats' file to flush each output line regarding training.
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
train_stats = ae.train(all_pc_data, conf, log_file=fout)
fout.close()

Get a batch of reconstuctions and their latent-codes.

In [ ]:
feed_pc, feed_model_names, _ = all_pc_data.next_batch(10)
reconstructions = ae.reconstruct(feed_pc)[0]
latent_codes = ae.transform(feed_pc)

Use any plotting mechanism such as matplotlib to visualize the results.

In [ ]:
i = 2
plot_3d_point_cloud(reconstructions[i][:, 0], 
                    reconstructions[i][:, 1], 
                    reconstructions[i][:, 2], in_u_sphere=True);

i = 4
plot_3d_point_cloud(reconstructions[i][:, 0], 
                    reconstructions[i][:, 1], 
                    reconstructions[i][:, 2], in_u_sphere=True);